# Integrações com o Flavor Network - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Importando Tabelas dos dados originais do Flavor Network

### Padronizando ingredientes

Com os arquivos criados no notebook anterior, agora é possível padronizar os ingredientes com os nomes do FooDB. Nesse caso, o arquivo de associação será utilizado. Criando as tabelas necessárias

In [2]:
DROP TABLE IF EXISTS AssociacaoComFooDB;
DROP TABLE IF EXISTS DadosFlavor;

CREATE TABLE AssociacaoComFooDB (
  Nome VARCHAR(100),
  FooDB VARCHAR(100),
  PRIMARY KEY (Nome)
) AS SELECT
  nome_ingrediente, nome_foodb
FROM CSVREAD('../data/external/associacaofinal.csv');

CREATE TABLE DadosFlavor (
  IDReceita INT,
  Banco VARCHAR(20),
  Ingrediente VARCHAR(100),
  Quantidade INT,
  Unidade VARCHAR(20),
  PRIMARY KEY (IDReceita, Banco, Ingrediente)
) AS SELECT
  IDReceita, Banco, Ingrediente, Quantidade, Unidade
FROM CSVREAD('../data/interim/flavor network/ingredientesdasreceitas.csv');


Visualizando os dados:

In [3]:
SELECT * FROM AssociacaoComFooDB LIMIT 10;


In [4]:
SELECT * FROM DadosFlavor LIMIT 10;


Criando uma VIEW com os nomes dos ingredientes padronizados:

In [5]:
DROP VIEW IF EXISTS Padronizados;

CREATE VIEW Padronizados AS
SELECT D.IDReceita, D.Banco, A.FooDB Ingrediente, D.Quantidade, D.Unidade
FROM DadosFlavor D, AssociacaoComFooDB A
WHERE D.Ingrediente = A.Nome
GROUP BY D.IDReceita, D.Banco, Ingrediente, D.Quantidade, D.Unidade
ORDER BY D.IDReceita;


Visualizando a VIEW:

In [6]:
SELECT * FROM Padronizados LIMIT 10;


Exportando esses dados:

In [7]:
CALL CSVWRITE('../data/interim/flavor network/ingredientesdasreceitaspadronizados.csv',
              'SELECT * FROM Padronizados');


457461

### Dados dos sabores

Com essas alterações do notebook anterior nos nomes dos ingredientes, os nomes deles nos sabores também precisam ser alterados. Como os sabores relacionam ingredientes presentes na base com ingredientes de fora, os ingredientes que estiverem na base original devem ter os nomes atualizados. Importando os dados dos sabores:

In [8]:
DROP TABLE IF EXISTS DadosSabores;

CREATE TABLE DadosSabores (
  Id INT,
  A VARCHAR(100),
  B VARCHAR(100),
  Quantidade INT NULL,
  PRIMARY KEY (Id)
) AS SELECT
  idsabor,ingredientea, ingredienteb, quantidade
FROM CSVREAD('../data/interim/flavor network/sabores.csv');


Visualizando:

In [9]:
SELECT * FROM DadosSabores LIMIT 10;


Alterando os nomes dos ingredientes das receitas para os valores novos:

In [10]:
UPDATE DadosSabores S
SET S.A = (SELECT A.FooDB FROM AssociacaoComFooDB A WHERE A.Nome = S.A)
WHERE S.A IN (SELECT DISTINCT DF.Ingrediente FROM DadosFlavor DF);

UPDATE DadosSabores S
SET S.B = (SELECT A.FooDB FROM AssociacaoComFooDB A WHERE A.Nome = S.B)
WHERE S.B IN (SELECT DISTINCT DF.Ingrediente FROM DadosFlavor DF);


Exportando os sabores finais, com quantidade associada:

In [11]:
CALL CSVWRITE('../data/interim/flavor network/saboresfinais.csv',
              'SELECT * FROM DadosSabores');


221777

### Origem das receitas

Seguindo o modelo proposto, é preciso normalizar também os nomes dos lugares de origem das receitas.

In [12]:
DROP TABLE IF EXISTS Receitas;

CREATE TABLE Receitas (
  Id INT,
  Origem VARCHAR(50),
  PRIMARY KEY (Id)
) AS SELECT
  IDReceita, Origem
FROM CSVREAD('../data/interim/flavor network/receitas.csv');


Visualizando:

In [13]:
SELECT * FROM Receitas LIMIT 10;


Selecionando apenas as regiões únicas:

In [14]:
DROP VIEW IF EXISTS Regioes;

CREATE VIEW Regioes AS
    SELECT DISTINCT Origem
    FROM Receitas
    ORDER BY Origem;


Visualizando:

In [15]:
SELECT * FROM Regioes LIMIT 10;


Exportando:

In [16]:
CALL CSVWRITE('../data/interim/flavor network/origens.csv',
              'SELECT * FROM Regioes');


11